# Demostrate interaction of python native objects List[Dict[str,Any]] with omnikeeper trait results

In [8]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import uuid

In [9]:
trait_id = "python_client_demo.test"
layer_id = "testlayer"

create a omnikeeper client

In [10]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [11]:
ret = okc.upsert_trait(okapiclient, TraitDefinition(trait_id, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))
assert(ret)

ensure layer exists

In [12]:
ret = okc.create_layer(okapiclient, layer_id, "just a test layer", okc.hex_string_to_rgb_color("#6666FF"))
assert(ret)

Create the input data in format of **input : List[Dict[str,Any]]**

In [13]:
initial_data = [
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
]
initial_data

[{'id': 1, 'array': ['a', 'b'], 'some_key': 'Value 1'},
 {'id': 3, 'array': ['c', 'd'], 'some_key': 'another Value'}]

write data to layer ``testlayer``

In [14]:
ret = okc.bulk_replace_trait_entities_by_filter(okapiclient, trait_id=trait_id, input=initial_data, id_attributes=["id"], id_relations=[], write_layer=layer_id, filter={})
assert(ret)

load data again from omnikeeper

In [15]:
updated_data = okc.get_all_traitentities(okapiclient, trait_id=trait_id, layers=[layer_id])
updated_data

[{'ciid': '97a3ed10-488c-4168-a194-22c25d7c8593',
  'id': 1,
  'array': ['a', 'b'],
  'some_key': 'Value 1'},
 {'ciid': 'c4ada9d4-a4eb-438c-b533-27fe62ef6460',
  'id': 3,
  'array': ['c', 'd'],
  'some_key': 'another Value'}]

modify the fetched traitentities

In [16]:
# change things in the returned data
# change single value of existing data
updated_data[0]["array"] = ["a", "b", "z"]

# add a new row, using our own created CIID
updated_data.append({"ciid": str(uuid.uuid4()), "id": 4, "array": ["x"], "some_key": "the new guy"})

# drop a row
updated_data.pop(1)
updated_data

[{'ciid': '97a3ed10-488c-4168-a194-22c25d7c8593',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'},
 {'ciid': '5ab89cf8-6c99-40f0-8738-21fe24fc2a0f',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'}]

write it back

In [17]:
ret = okc.bulk_replace_trait_entities(okapiclient, trait_id=trait_id, input=updated_data, write_layer=layer_id)
assert(ret)

Fetch again the data

In [18]:
final_data = okc.get_all_traitentities(okapiclient, trait_id=trait_id, layers=[layer_id])
final_data

[{'ciid': '5ab89cf8-6c99-40f0-8738-21fe24fc2a0f',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'},
 {'ciid': '97a3ed10-488c-4168-a194-22c25d7c8593',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'}]